In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from datetime import datetime

In [2]:
spark = SparkSession.\
        builder.\
        master('local[*]').\
        appName('hw_2023').\
        getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


26/01/13 06:05:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/13 06:05:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/13 06:05:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# Question 1: Execute spark.version, what is the version?

spark.version

#### Answer: 3.3.2

'3.3.2'

In [4]:
# Question 2: HVFHW June 2021

# Read it with Spark using the same schema as we did in the lessons.
# We will use this dataset for all the remaining questions.
# Repartition it to 12 partitions and save it to parquet.
# What is the average size of the Parquet
# (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

!curl -Lf https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz -o 2023.csv.gz
!gzip -d 2023.csv.gz
csv_df = spark.read.option('header', 'true').option('inferSchema', 'true').csv('2023.csv')
csv_df.repartition(12).write.parquet('hw_2023/')
!rm -f 2023/.par*
!ls -lash hw_2023/

### Answer: 23

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  167M  100  167M    0     0  28.1M      0  0:00:05  0:00:05 --:--:-- 33.4M


[Stage 4:====================================================>    (11 + 1) / 12]

total 273M
4.0K drwxr-xr-x 2 ajay ajay 4.0K Jan 13 06:07 .
4.0K drwxrwxr-x 4 ajay ajay 4.0K Jan 13 06:06 ..
4.0K -rw-r--r-- 1 ajay ajay    8 Jan 13 06:07 ._SUCCESS.crc
184K -rw-r--r-- 1 ajay ajay 181K Jan 13 06:06 .part-00000-31fe35af-431e-41e2-b0ed-0993090cc564-c000.snappy.parquet.crc
184K -rw-r--r-- 1 ajay ajay 181K Jan 13 06:06 .part-00001-31fe35af-431e-41e2-b0ed-0993090cc564-c000.snappy.parquet.crc
184K -rw-r--r-- 1 ajay ajay 181K Jan 13 06:06 .part-00002-31fe35af-431e-41e2-b0ed-0993090cc564-c000.snappy.parquet.crc
184K -rw-r--r-- 1 ajay ajay 181K Jan 13 06:06 .part-00003-31fe35af-431e-41e2-b0ed-0993090cc564-c000.snappy.parquet.crc
184K -rw-r--r-- 1 ajay ajay 181K Jan 13 06:06 .part-00004-31fe35af-431e-41e2-b0ed-0993090cc564-c000.snappy.parquet.crc
184K -rw-r--r-- 1 ajay ajay 181K Jan 13 06:06 .part-00005-31fe35af-431e-41e2-b0ed-0993090cc564-c000.snappy.parquet.crc
184K -rw-r--r-- 1 ajay ajay 181K Jan 13 06:06 .part-00006-31fe35af-431e-41e2-b0ed-0993090cc564-c000.snappy.parquet.crc

In [5]:
# Question 3: Count records
# How many taxi trips were there on June 15?
# Consider only trips that started on June 15.

df = spark.read.parquet('hw_2023/*')
df.filter(F.to_date(df.pickup_datetime) == datetime(2021, 6, 15)).count()

#### Answer: 452470

452470

In [6]:
# Question 4: Longest trip for each day
# Now calculate the duration for each trip. How long was the longest trip in Hours?

df = df.withColumn('duration', df.dropoff_datetime - df.pickup_datetime)
df = df.withColumn('duration', F.expr("""
round(extract(day from duration) * 24 + 
extract(hour from duration) + 
extract(min from duration) / 60 + 
extract(second from duration) / 3600, 2)
"""))
df.sort('duration', ascending = False).select(F.to_date('pickup_datetime'), 'duration').limit(10).show()

#### Answer: 66.88

[Stage 9:>                                                          (0 + 4) / 4]

+------------------------+--------+
|to_date(pickup_datetime)|duration|
+------------------------+--------+
|              2021-06-25|   66.88|
|              2021-06-22|   25.55|
|              2021-06-27|   19.98|
|              2021-06-26|    18.2|
|              2021-06-23|   16.47|
|              2021-06-23|   14.27|
|              2021-06-24|   13.91|
|              2021-06-04|   11.67|
|              2021-06-27|   11.37|
|              2021-06-20|   10.98|
+------------------------+--------+



In [7]:
# Question 5: User Interface
# Spark’s User Interface which shows application's dashboard runs on which local port?

spark._activeSession

### Answer: currently running on 4042 but usally on 4040

In [8]:
# Question 6: Most frequent pickup location zone
# Using the zone lookup data and the fhvhv June 2021 data, what is the name of the most frequent pickup location zone?

!curl -Lf https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv -o z.csv
zone = spark.read.option('header', 'true').option('inferSchema','true').csv('z.csv')
joined = df.join(zone, zone.LocationID == df.PULocationID, 'left')
joined.groupby('Zone').count().sort('count', ascending = False).limit(5).show()

#### Answer: Crown Heights North

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12322  100 12322    0     0  25025      0 --:--:-- --:--:-- --:--:-- 25025


[Stage 14:>                                                         (0 + 4) / 4]

+-------------------+------+
|               Zone| count|
+-------------------+------+
|Crown Heights North|231279|
|       East Village|221244|
|        JFK Airport|188867|
|     Bushwick South|187929|
|      East New York|186780|
+-------------------+------+



In [9]:
spark.stop()
!rm -rf *.csv *.parquet hw_*